<a href="https://colab.research.google.com/github/HusseinElwakeel/Campus-Portal/blob/main/pattern_assignment_Hussein_Elwakeel_20190388.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from cv2 import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [30]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [31]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download 'kritikseth/fruit-and-vegetable-image-recognition' --unzip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
fruit-and-vegetable-image-recognition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [32]:
val_path="validation"
train_path="train"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_path,
                                                               seed=2509,
                                                               image_size=(224, 224),
                                                              batch_size=32)
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(val_path,
                                                              seed=2509,
                                                              image_size=(224, 224),
                                                              shuffle=False,
                                                              batch_size=32)

Found 133 files belonging to 2 classes.


FileNotFoundError: ignored

In [ ]:
class_names = train_dataset.class_names
print(len(class_names))

In [ ]:
class_names

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(len(class_names),activation='softmax'))

In [ ]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(lr=0.001),
    metrics = ["accuracy"])
model.summary()

In [ ]:
history = model.fit(x=train_dataset,
                    epochs= 50,
                    validation_data=val_dataset)

In [ ]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel('Time')
plt.legend(['val_loss', 'loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Epochs")
plt.xlabel('Time')
plt.legend(['val_accuracy', 'acc'], loc='upper left')
plt.show()

In [ ]:
from keras_preprocessing import image
def show_model(image_path):
  img = image.load_img(image_path, target_size=(224,224,3))
  plt.imshow(img)
  plt.show()
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  pred = model.predict(images, batch_size=32)
  label = np.argmax(pred)
  print("Actual: "+image_path.split("/")[-2])
  print("Predicted: "+class_names[label])

show_model("test/cabbage/Image_7.jpg")

In [ ]:
show_model("test/corn/Image_10.jpg")

In [ ]:
show_model("test/banana/Image_5.jpg")

In [ ]:
show_model("test/tomato/Image_2.jpg")

In [ ]:
model.save("model.h5")

In [ ]:
model= tf.keras.models.load_model(os.path.join('drive','MyDrive','model.h5'))

In [ ]:
tf.keras.utils.plot_model(model, to_file="model.png", show_shapes=True)

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
food_name=str(class_names[label])
food_name

In [ ]:
html=requests.get('https://www.fatsecret.com/calories-nutrition/usda/'+food_name+'s')

In [ ]:
soup = BeautifulSoup(html.content,"html.parser")
# print(soup.prettify())

In [ ]:
[(i.text,count) for count,i in enumerate(soup.find_all("td")) ]

In [ ]:
h=soup.find_all("td")[16].text.strip().split('\n')[1]

In [ ]:
print("the calories in "+str(class_names[label])+'is'+str(h))